## Remote Sensing in Environment (2024 Spring) - Midterm

Question 2

### 1. Reading the ENVI Spectral Library

In [ ]:
from SpectralLib import SpectralLib
from Mix import Mixer, MixerCollection

ABSORPTION = [(1350,1400), (1830,1930)]
_PV = SpectralLib('ASD_PV_RM.lib', 'ASD_PV_RM.HDR', ABSORPTION)
_NPV = SpectralLib('ASD_NPV_RM.lib', 'ASD_NPV_RM.HDR', ABSORPTION)
_SOIL = SpectralLib('ASD_Soils_RM.lib', 'ASD_Soils_RM.HDR', ABSORPTION)
PV = _PV.copy()
NPV = _NPV.copy()
SOIL = _SOIL.copy()
PV._rm = [(300, 800)]
NPV._rm = [(300, 800)]
SOIL._rm = [(300, 800)]

### 2.1 Read Landsat 9 OLI-2 Band Specifications - Summary

Example outputs:
|Band Title|FPM No.|Wavelength Lower Limit (nm)|Wavelength Upper Limit (nm)|
|----------|------:|--------------------------:|--------------------------:|
|CA        |      1|                    434.977|                    450.466|
|CA        |      2|                    435.120|                    450.563|
|       ...|    ...|                        ...|                        ...|
|Blue      |      1|                    451.751|                    511.797|
|       ...|    ...|                        ...|                        ...|
|Cirrus    |     14|                   1363.416|                   1384.279|

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

source_s = pd.read_excel('L9_OLI2_Ball_FPM_RSR.v1.0.xlsx', sheet_name = 'Band summary', header = [0, 1])
source_r = pd.read_excel('L9_OLI2_Ball_FPM_RSR.v1.0.xlsx', sheet_name = [2, 4, 5, 6, 7, 9, 11, 12, 8])  # CA, Blue, Green, Red, NIR, SWIR1, SWIR2, Pan, Cirrus

In [ ]:
band_order = ['CA', 'Blue', 'Green', 'Red', 'NIR', 'SWIR1', 'SWIR2', 'Pan', 'Cirrus']
band_title = source_s['Band'].iloc[:, 0]
fpms = source_s['Band'].iloc[:, 1]
lower_limit = source_s['FWHM wavelength [nm]'].iloc[:, 0]
upper_limit = source_s['FWHM wavelength [nm]'].iloc[:, 1]
summary = pd.concat([band_title, fpms, lower_limit, upper_limit], axis = 1)
summary.columns = ['Band', 'FPM', 'Lower', 'Upper']
summary['Band'] = pd.Categorical(summary['Band'], band_order)
summary.sort_values(['Band', 'FPM'], ignore_index = True, inplace = True)
display(summary)
 
limits = summary.groupby('Band', observed = True).mean().iloc[:, 1:].reset_index()
fig, ax = plt.subplots()
fig.set_size_inches(10, 2)
ax.set(xlabel = 'Wavelength (nm)', xlim = (400, 2500), ylim = (0, 0.155), xticks = range(300, 2700, 200), yticks = [], title = 'Landsat 9 OLI-2 Band Specifications')
ax.spines[['top', 'left', 'right']].set(visible = False)
ax.set_prop_cycle(color = ['dodgerblue', 'mediumblue', 'green', 'red', 'brown', 'goldenrod', 'gold', 'grey', 'orange'])
for k, band in limits.iterrows():
    if (band['Band'] not in ['CA', 'Pan', 'Cirrus']):
        ax.fill_betweenx([0.04, 0.07], band['Lower'], band['Upper'], alpha = 0.3, label = band['Band'])
        ax.text((band['Lower'] + band['Upper']) / 2, 0.055, k + 1, ha = 'center', va = 'center', size = 8)

    else:
        ax.fill_betweenx([0.08, 0.11], band['Lower'], band['Upper'], alpha = 0.3, label = band['Band'])
        ax.text((band['Lower'] + band['Upper']) / 2, 0.095, k + 1, ha = 'center', va = 'center', size = 8)

    ax.fill_betweenx([0.01, 0.02], band['Lower'], band['Upper'], fc = 'dimgrey')

ax.text(2480, 0.095, '≥ L8', ha = 'right', va = 'center', size = 8)
ax.text(2480, 0.055, '≥ L4', ha = 'right', va = 'center', size = 8)
ax.text(2480, 0.015, 'Total', ha = 'right', va = 'center', size = 8)
ax.legend(ncols = len(limits), mode = 'expand')
plt.show()

### 2.2 Read Landsat 9 OLI-2 Band Specifications - Relative Response

In [ ]:
def oli2_reflectance(obj: SpectralLib, source: dict, clip: bool = True, c_range: list = None, join: bool = True, use_band_name: bool = False, **kwargs) -> pd.DataFrame:
    '''
    Simulate the reflectance Landsat 9 OLI-2 sensors received in the space. 
    '''
    response = [v.set_index('wavelength', drop = True).iloc[:, :14] for k, v in source.items()]
    response = [item.apply(np.mean, axis = 1) for item in response]
    band_order = ['CA', 'Blue', 'Green', 'Red', 'NIR', 'SWIR1', 'SWIR2', 'Pan', 'Cirrus']
    default_clip = [(435, 451), (452, 512), (533, 589), (636, 673), (850, 879), (1565, 1651), (2105, 2295), (503, 676), (1364, 1385)]
    clip_range = [c_range[i] if c_range else (l, u) for i, (l, u) in enumerate(default_clip)]
    band_reflectance = [obj.data.loc[response[i].index, :].multiply(response[i], axis = 0).fillna(0) for i in range(len(response))]
    band_reflectance = [band.loc[clip_range[i][0]:clip_range[i][1] + 1, :] if clip else band for i, band in enumerate(band_reflectance)]
    if not(join):
        simulated = {band_order[i]: band_reflectance[i] for i in range(len(band_order))}
        return simulated
    
    else:
        proportions = [band.apply(lambda r: r / band.sum()) for band in response]
        band_totals = [band_reflectance[i].multiply(proportions[i][band_reflectance[i].index], axis = 0).sum() for i in range(len(proportions))]
        if use_band_name:
            band_summary = pd.DataFrame({band_order[i]: band_totals[i] for i in range(len(band_order))}).T
        
        else:
            band_center  = [round((l + u) / 2) for l, u in clip_range]
            band_summary = pd.DataFrame({band_center[i]: band_totals[i] for i in range(len(band_order))}).T

        return band_summary
    
def oli2_plots(obj: SpectralLib, source: dict, name: str = '', primary_color: list[str] = ['tab:blue', 'tab:orange', 'tab:green'], secondary_color: str = 'grey', **kwargs) -> None:
    '''
    Give the simulated Landsat 9 OLI-2 reflectance plots.
    '''
    if isinstance(obj, SpectralLib):
        data     = oli2_reflectance(obj, source, **kwargs).sort_index().T
        original = obj.stat
        stats = data.describe().T.iloc[:, [1, 4, 5, 6]]
        fig, ax = plt.subplots()
        ax.set(xlabel = 'Wavelength (nm)', ylabel = 'Reflectance', ylim = (0, 0.5), title = f'Simulating {name} Spectra Distibution from OLI-2')
        ax.plot(stats.index, stats['mean'], color = secondary_color, ls = 'dashed', lw = 1, label = 'Mean')
        ax.plot(stats.index, stats['50%'], color = primary_color[0], lw = 1.5, label = 'Median')
        ax.plot(original.index, original['50%'], color = primary_color[0], ls = 'dotted', lw = 1, label = 'Hyper. Median')
        ax.scatter(stats.index, stats['50%'], fc = primary_color[0], s = 10, zorder = 2.1)
        ax.scatter(stats.index, stats['mean'], fc = secondary_color, s = 10, zorder = 2)
        ax.fill_between(stats.index, stats['25%'], stats['75%'], facecolor = primary_color[0], alpha = 0.3, label = 'IQR Range')
        ax.legend()
        ax.grid()
        plt.show()

    elif isinstance(obj, dict):
        counter = 0
        fig, ax = plt.subplots()
        ax.set(xlabel = 'Wavelength (nm)', ylabel = 'Reflectance', xlim = (250, 2600), ylim = (0, 0.5), title = f'Simulating All Spectra Distibution from OLI-2')
        for s_name, item in obj.items():
            if isinstance(item, SpectralLib):
                data  = oli2_reflectance(item, source, **kwargs).sort_index().T
                stats = data.describe().T.iloc[:, [1, 4, 5, 6]]
                ax.plot(stats.index, stats['50%'], color = primary_color[counter % len(primary_color)], lw = 1.5, label = f'{s_name} Median')
                ax.scatter(stats.index, stats['50%'], fc = primary_color[counter % len(primary_color)], s = 10, zorder = 2)
                ax.fill_between(stats.index, stats['25%'], stats['75%'], facecolor = primary_color[counter % len(primary_color)], alpha = 0.3)
                counter += 1
        
        ax.fill_between([300, 2500], pd.Series(0, index = [300, 2500]), pd.Series(0, index = [300, 2500]), facecolor = 'grey', alpha = 0.3, label = 'IQR Range')
        ax.legend()
        ax.grid()
        plt.show()

In [ ]:
PV.data = oli2_reflectance(_PV, source_r).sort_index()
NPV.data = oli2_reflectance(_NPV, source_r).sort_index()
SOIL.data = oli2_reflectance(_SOIL, source_r).sort_index()
PV = PV.differential()
NPV = NPV.differential()
SOIL = SOIL.differential()

oli2_plots(_PV, source_r, 'PV', ['forestgreen'])
oli2_plots(_NPV, source_r, 'NPV', ['coral'])
oli2_plots(_SOIL, source_r, 'SOIL', ['olive'])
oli2_plots({'PV': _PV, 'NPV': _NPV, 'Soil': _SOIL}, source_r, primary_color = ['forestgreen', 'coral', 'olive'])

### 3. Answer the Question
Can we use Landsat 9 OLI-2 Data to estimate fractional NPV cover? Let's find out.

In [ ]:
# Using the solution used in Question 1
PV.data = oli2_reflectance(_PV, source_r).sort_index()
NPV.data = oli2_reflectance(_NPV, source_r).sort_index()
SOIL.data = oli2_reflectance(_SOIL, source_r).sort_index()
PV = PV.differential()
NPV = NPV.differential()
SOIL = SOIL.differential()
specs = ['PV', 'NPV', 'Soil']

In [ ]:
err = (PV.stat['std'] ** 2 + NPV.stat['std'] ** 2 + SOIL.stat['std'] ** 2) ** (1 / 2)
MIX = MixerCollection(PV.stat['50%'], NPV.stat['50%'], SOIL.stat['50%'], err)
T011 = MIX.ingredient_plots(PV, specs, 1, weight_factor = 2)
T012 = MIX.ingredient_plots(NPV, specs, 2, weight_factor = 2)
T013 = MIX.ingredient_plots(SOIL, specs, 3, weight_factor = 2)
Mixer.error_plot(T011, T012, T013, {0: 'PV', 1: 'NPV', 2: 'Soil', 3: 'Ambi.'})

In [ ]:
rdmSpectra = Mixer.get_random_spectra_mix(3000, PV, NPV, SOIL, seed = 123)
MIX.comparing_plots(rdmSpectra, 'NPV', status = True)